In [1]:
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import sklearn


from sklearn.preprocessing import normalize
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.cross_validation import StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

import utils_veolia


/Users/christophelanternier/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
input_raw, output_raw, input_submission = utils_veolia.load_data()

In [3]:
input_preprocessed = utils_veolia.preprocess(input_raw, year=2014, more_features=False)

In [4]:
input_train, output_train, input_test, output_test = utils_veolia.split_train_test_Kfold(output_raw, input_preprocessed)

In [5]:
input_train_duplicate, output_train_duplicate = utils_veolia.data_augmentation_basic(input_train, output_train, repetitions = 6)

# Classification

In [199]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

In [200]:
RFC = RandomForestClassifier(n_estimators=10, max_depth=20)
GBC = GradientBoostingClassifier(n_estimators=10, max_depth=20)
ABC = AdaBoostClassifier(n_estimators=20)

In [201]:
predict_proba = True
YEAR = 2014
input_preprocessed = utils_veolia.preprocess(input_raw, year=YEAR, more_features=True)
input_train, output_train, input_test, output_test = utils_veolia.split_train_test_Kfold(output_raw, input_preprocessed)
input_train_duplicate, output_train_duplicate = utils_veolia.data_augmentation_basic(input_train, output_train, year = YEAR,repetitions = 8)

X_train = input_train_duplicate
utils_veolia.print_repartition(output_train_duplicate)
Y_train = utils_veolia.preprocess_output(output_train_duplicate, year = YEAR)
X_test = input_test
Y_test_2014 = utils_veolia.preprocess_output(output_test, year = YEAR)

rdm_1 = ABC
rdm_1.fit(X_train, Y_train)

if predict_proba:
    y_pred_2014 = rdm_1.predict_proba(X_test)
    y_pred_2014 = y_pred_2014[:,1]
else:
    y_pred_2014 = rdm_1.predict(X_test)
    print(classification_report(Y_test_2014,y_pred_2014))

Repartition train: 
2015:  529
2014:  6912
Not Broken:  9670


In [202]:
YEAR = 2015
input_preprocessed = utils_veolia.preprocess(input_raw, year=YEAR, more_features=True)
input_train, output_train, input_test, output_test = utils_veolia.split_train_test_Kfold(output_raw, input_preprocessed)
input_train_duplicate, output_train_duplicate = utils_veolia.data_augmentation_basic(input_train, output_train, year = YEAR,repetitions = 8)

X_train = input_train_duplicate
utils_veolia.print_repartition(output_train_duplicate)
Y_train = utils_veolia.preprocess_output(output_train_duplicate, year = YEAR)
X_test = input_test
Y_test_2015 = utils_veolia.preprocess_output(output_test, year = YEAR)

rdm_1 = GBC
rdm_1.fit(X_train, Y_train)

if predict_proba:
    y_pred_2015 = rdm_1.predict_proba(X_test)
    y_pred_2015 = y_pred_2015[:,1]
else:
    y_pred_2015 = rdm_1.predict(X_test)
    print(classification_report(Y_test_2015,y_pred_2015))

Repartition train: 
2015:  4352
2014:  539
Not Broken:  9670


In [203]:
pred = np.array([y_pred_2014,y_pred_2015]).T
true = np.array([Y_test_2014,Y_test_2015]).T

utils_veolia.score_function(true, pred)

0.7003855546348241